### Training
---

In [ ]:
from typing import Dict, List, Optional

from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager
from nemo.utils import logging
from nemo.collections.nlp.parts.utils_funcs import tensor2list

import os
import time

import torch
import pytorch_lightning as pl

from omegaconf import OmegaConf

DOCKER = False

if DOCKER:
    HOME_DIR = "/workspace"
else:
    HOME_DIR = "/home/rchen/"

# directory with data converted to nemo format
data_dir = os.path.join(HOME_DIR, "multiatis")

run_name = "test"
config_file = os.path.join("/home/rchen/NeMo/examples/nlp/multi-label-intent-slot-classification/conf/multi-label-intent-slot-classification.yaml")
config = OmegaConf.load(config_file)
config.model.data_dir = data_dir
config.model.validation_ds.prefix = "dev"
config.model.test_ds.prefix = "dev"
config.model.intent_loss_weight = 0.6
config.model.class_balancing = "weighted_loss"
config.model.head.num_output_layers = 2
config.trainer.max_epochs = 5


# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda

# config.trainer.precision = 16 if torch.cuda.is_available() else 32
config.trainer.precision = 32

# for mixed precision training, uncomment the line below (precision should be set to 16 and amp_level to O1):
# config.trainer.amp_level = 'O1'

# remove distributed training flags
config.trainer.accelerator = None

# early_stop_callback = EarlyStopping(monitor='intent_f1', min_delta=1e-1, patience=10, verbose=True, mode='max')

trainer = pl.Trainer(**config.trainer)
config.exp_manager.exp_dir = os.path.join(HOME_DIR, "output/" + run_name)
config.exp_manager.create_checkpoint_callback = True
config.exp_manager.version = time.strftime('%Y-%m-%d_%H-%M-%S')

exp_dir = exp_manager(trainer, config.get("exp_manager", None))
model = nemo_nlp.models.MultiLabelIntentSlotClassificationModel(config.model, trainer=trainer)
trainer.fit(model)

### Evaluation
---

In [3]:
# extract the path of the best checkpoint from the training, you may update it to any other saved checkpoint file
checkpoint_path = trainer.checkpoint_callback.best_model_path

eval_model= nemo_nlp.models.MultiLabelIntentSlotClassificationModel.restore_from('/home/rchen/output/test/MultiLabelIntentSlot/2022-01-21_08-38-23/checkpoints/MultiLabelIntentSlot.nemo')

# load the model from this checkpoint
#eval_model = nemo_nlp.models.MultiLabelIntentSlotClassificationModel.load_from_checkpoint(checkpoint_path=checkpoint_path)

[NeMo I 2022-01-21 09:24:35 tokenizer_utils:125] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /tmp/tmpuel7u8uz/c6e42b8a7b054607bbd3ceac523f4514_vocab.txt, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.


[NeMo I 2022-01-21 09:24:39 multi_label_intent_slot_classification_descriptor:87]  Stats calculating for train mode...
[NeMo I 2022-01-21 09:24:39 multi_label_intent_slot_classification_descriptor:113] Three most popular intents in train mode:
[NeMo I 2022-01-21 09:24:39 data_preprocessing:194] label: (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 3309 out of 4478 (73.89%).
[NeMo I 2022-01-21 09:24:39 data_preprocessing:194] label: (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 385 out of 4478 (8.60%).
[NeMo I 2022-01-21 09:24:39 data_preprocessing:194] label: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0), 230 out of 4478 (5.14%).
[NeMo I 2022-01-21 09:24:39 multi_label_intent_slot_classification_descriptor:119] Three most popular slots in train mode:
[NeMo I 2022-01-21 09:24:39 data_preprocessing:194] label: 101, 32066 out of 50497 (63.50%).
[NeMo I 2022-01-21 09:24:39 data_preprocessing:194] label: 4, 3919 out of 50497 (7.76%).
[NeMo I 2022-01-21 09:24:39 data_prepr

[NeMo I 2022-01-21 09:24:39 multi_label_intent_slot_classification_descriptor:87]  Stats calculating for dev mode...
[NeMo I 2022-01-21 09:24:39 multi_label_intent_slot_classification_descriptor:113] Three most popular intents in dev mode:
[NeMo I 2022-01-21 09:24:39 data_preprocessing:194] label: (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 357 out of 500 (71.40%).
[NeMo I 2022-01-21 09:24:39 data_preprocessing:194] label: (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), 39 out of 500 (7.80%).
[NeMo I 2022-01-21 09:24:39 data_preprocessing:194] label: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0), 25 out of 500 (5.00%).
[NeMo I 2022-01-21 09:24:39 multi_label_intent_slot_classification_descriptor:119] Three most popular slots in dev mode:
[NeMo I 2022-01-21 09:24:39 data_preprocessing:194] label: 101, 3604 out of 5703 (63.19%).
[NeMo I 2022-01-21 09:24:39 data_preprocessing:194] label: 43, 434 out of 5703 (7.61%).
[NeMo I 2022-01-21 09:24:39 data_preprocessing:194] l

[NeMo W 2022-01-21 09:24:39 modelPT:136] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    prefix: train
    batch_size: 32
    shuffle: true
    num_samples: -1
    num_workers: 4
    drop_last: false
    pin_memory: false
    
[NeMo W 2022-01-21 09:24:39 modelPT:143] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    prefix: dev
    batch_size: 32
    shuffle: false
    num_samples: -1
    num_workers: 2
    drop_last: false
    pin_memory: false
    
[NeMo W 2022-01-21 09:24:39 modelPT:149] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a valid configuration file to setup the test data loader(s).
    Test 

WEIGHTED LOSS
[NeMo I 2022-01-21 09:24:43 save_restore_connector:149] Model MultiLabelIntentSlotClassificationModel was successfully restored from /home/rchen/output/test/MultiLabelIntentSlot/2022-01-21_08-38-23/checkpoints/MultiLabelIntentSlot.nemo.


In [ ]:
eval_model

In [4]:
queries = [
    'i would like to find a flight from charlotte to las vegas that makes a stop in st. louis',
    'on april first i need a ticket from tacoma to san jose departing before 7 am',
    'how much is the limousine service in boston',
    'How do I fly to China'
]

pred_intents, pred_slots = eval_model.predict_from_examples(queries, config.model.test_ds)
logging.info('The prediction results of some sample queries with the trained model:')

    
for query, intent in zip(queries, pred_intents):
    logging.info(f'Query : {query}')
    logging.info(f'Predicted Intent: {intent}')


[NeMo I 2022-01-21 09:24:51 multi_label_intent_slot_classification_dataset:92] Setting max length to: 22
[NeMo I 2022-01-21 09:24:51 data_preprocessing:358] Some stats of the lengths of the sequences:
[NeMo I 2022-01-21 09:24:51 data_preprocessing:364] Min: 8 |                  Max: 22 |                  Mean: 14.5 |                  Median: 14.0
[NeMo I 2022-01-21 09:24:51 data_preprocessing:366] 75 percentile: 19.00
[NeMo I 2022-01-21 09:24:51 data_preprocessing:367] 99 percentile: 21.88
[NeMo I 2022-01-21 09:24:51 multi_label_intent_slot_classification_dataset:121] 0 are longer than 22
Intent Predictions:
[[0.013444727286696434, 0.002968158107250929, 0.002989558270201087, 0.0038464025128632784, 0.005968357436358929, 0.9796890616416931, 0.012039211578667164, 0.003440923523157835, 0.006254768930375576, 0.003423362970352173, 0.005885966122150421, 0.006707846187055111, 0.002850351156666875, 0.0035031591542065144, 0.007512572221457958, 0.004105986561626196, 0.004516006913036108], [0.3433

In [ ]:
print(config)

In [ ]:
config.keys()

In [ ]:
checkpoint_path

In [ ]:
print(config_file)

In [ ]:
config = OmegaConf.load(config_file)

In [ ]:
checkpoint_path

In [ ]:
!ls /home/rchen/output/test/MultiLabelIntentSlot/

In [ ]:
!ls /home/rchen/output/test/MultiLabelIntentSlot/2022-01-20_11-28-12/checkpoints/MultiLabelIntentSlot.nemo

### Torch Tensor Experiments 
---

In [ ]:
import torch

In [ ]:
lst = [[1,3,1], [4,5,6], [7,8,9]]

In [ ]:
a = torch.tensor(lst)

In [ ]:
a

In [ ]:
a.t()

In [ ]:
a.t()[0]

In [ ]:
for x in a.t()[0]:
    print(x == torch.tensor(1))

In [ ]:
a

In [ ]:
bst = [[1,2,1], [1,5,1], [1,8,1]]

In [ ]:
b = torch.tensor(bst)

In [ ]:
current_label = a[0]
labels_predicted = b[0]

In [ ]:
current_label

In [ ]:
labels_predicted

In [ ]:
(current_label == labels_predicted)[current_label == 1].sum()

In [ ]:
(labels_predicted[current_label] == current_label[labels_predicted]

In [ ]:
current_label

In [ ]:
labels_predicted

In [ ]:
a[0]

In [ ]:
a[0][current_label]

In [ ]:
labels_predicted[current_label]

In [ ]:
current_label[labels_predicted]

In [ ]:
l = [1,2,3]
c = [1,4,3]
d = torch.tensor(l) == 1
e = torch.tensor(c) == 1

In [ ]:
d == 1

In [ ]:
e == 1

In [ ]:
(d == e)[d]

In [ ]:
e

In [ ]:
d